Import Necessary Libraries

In [119]:
import statsapi as api
import pandas as pd
import tempfile
import os
import re
import requests
from bs4 import BeautifulSoup, SoupStrainer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


Input Player Name and Year

In [120]:
first_name = input("Enter Player First Name: ")
last_name = input('Enter Player Last Name: ')
year = input(f'Enter Year: ')
player_name = f'{first_name.lower()} {last_name.lower()}'


fetch player_id 

In [121]:
player_info = api.lookup_player(player_name, season=year)
player_id = player_info[0]['id']
team_id = player_info[0]['currentTeam']['id']
print(player_id)
print(team_id)

661388
158


Gather Player hand

In [122]:
person_info = api.get('people',{'personIds':player_id, 'season':year,'sportId':1})
batter_hand = person_info['people'][0]['batSide']['code']
print(batter_hand)

R


fetch game_pk, home, opp_id, venue_id, sPitcher_id, sPitcher_hand

In [123]:
date = '2024-04-20'
game_info = api.schedule(date=date,team=team_id,sportId=1)
game_pk = game_info[0]['game_id']
home_team_id = game_info[0]['home_id']
if home_team_id == team_id:
    home = True
    opp_id = game_info[0]['away_id']
    sPitcher_name = game_info[0]['away_probable_pitcher']
else:
    home = False
    opp_id = game_info[0]['home_id']
    sPitcher_name = game_info[0]['home_probable_pitcher']

sPitcher_info = api.lookup_player(sPitcher_name)
sPitcher_id = sPitcher_info[0]['id']
sPitcher_info_2 = api.get('people',{'personIds':sPitcher_id, 'season':year,'sportId':1})
sPitcher_hand = sPitcher_info_2['people'][0]['pitchHand']['code']

venue_id = game_info[0]['venue_id']

if game_info[0]['status'] == 'Final':
    game_status = True
else:
    game_status = False

print(game_status)
print(game_pk)
print(opp_id)
print(sPitcher_id)
print(sPitcher_hand)
print(venue_id)
print(home)

True
745190
138
571945
R
2889
False


fetch player game stats: temp, wind_speed, hp_ump_id, game_time. Fetch list of proj_ump_ids if Home Plate Umpire not yet specified 

In [124]:
game_boxscore_info = api.boxscore_data(gamePk=game_pk)

for x in game_boxscore_info['gameBoxInfo']:
    if x.get('label') == 'Weather':
        temp_info = x['value']
        temps = re.findall(r'\d+', temp_info)
        if temps:
            temp = int(temps[0])

    if x.get('label') == 'Wind':
        wind_speed_info = x['value']
        winds = re.findall(r'\d+', wind_speed_info)
        if winds:
            wind_speed = int(winds[0])

    if x.get('label') == 'First pitch':
        game_time = x['value']

    if x.get('label') == 'Umpires':
        ump_info = x['value']
        if 'HP:' in ump_info:
            pattern = r'HP: (.*?)\.'
            match = re.search(pattern, ump_info)
            if match:
                hp_ump = match.group(1)
                hp_ump_info = api.get('jobs_umpires', {})
                for person in hp_ump_info['roster']:
                    if person['person'].get('fullName') == hp_ump:
                        hp_ump_id = person['person'].get('id')
                        print(hp_ump_id)

        else:
            umps = re.findall(r'[\w\s]+', ump_info)
            umps = [ump.strip() for ump in umps if ump.strip]
            ump_list = list(umps) + [None]*(4-len(umps))
            umps_tuple = tuple(ump_list[:4])
            proj_ump_ids = []
            for ump in umps_tuple:
                ump_info = api.get('jobs_umpires',{})
                for person in ump_info['roster']:
                    if person['person'].get('fullName') == ump:
                        ump_id = person['person'].get('id')
                        proj_ump_ids.append(ump_id)
            print(proj_ump_ids)

            

print(temp)
print(wind_speed)
print(game_time)


427413
58
11
1:16 PM.


fetch player game data: 
- batting position

In [125]:
game_content = api.boxscore_data(gamePk=game_pk)
if home == True:
    batters_info = game_content['homeBatters']
else:
    batters_info = game_content['awayBatters']

for batter in batters_info:
    if batter.get('personId') == player_id:
        batting_position = int(batter.get('battingOrder')[0])
        print (batting_position)


2


fetch game player stats
- If game_status == True
    - gather player stats
- If game_status == False
    - set player stats to None

In [126]:
if game_status == True:
    game_stats_info = api.get('person_stats',{'personId':player_id,'gamePk':game_pk})
    game_stats = game_stats_info['stats'][0]['splits'][2]['stat']
    ab = game_stats.get('atBats')
    h = game_stats.get('hits')
    r = game_stats.get('runs')
    rbi = game_stats.get('rbi')
    hr = game_stats.get('homeRuns')
    k = game_stats.get('strikeOuts')
    bb = game_stats.get('baseOnBalls')

else:
    ab = None
    h = None
    r = None
    rbi = None
    hr = None
    k = None
    bb = None


print(f'ab: {ab}')
print(f'h: {h}')
print(f'r: {r}')
print(f'rbi: {rbi}')
print(f'hr: {hr}')
print(f'k: {k}')
print(f'bb: {bb}')

    

ab: 6
h: 3
r: 2
rbi: 1
hr: 0
k: 1
bb: 0


Format player URL for finding streak averages

In [127]:
formatted_player_name = player_name.replace(' ', '-')
print(formatted_player_name)

streak_url = f'https://www.mlb.com/player/{formatted_player_name}-{player_id}'


william-contreras


Fetch Streak Averages Data

In [128]:
response = requests.get(streak_url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

div = soup.find('div', class_='player-splits--last player-splits--last-x')

header = div.find('thead').find('tr')

headers = {}

for th in header.find_all('th'):
    stat_index = th['data-index']
    stat_header = th.find('span')
    stat_name = stat_header.text
    headers[stat_index] = stat_name

table_row = div.find('tbody').find_all('tr')

data = []

for row in table_row:

    row_data = {}
    for td in row.find_all('td'):
        stat_type = td['data-col']
        stat = td.find('span')
        stat_value = stat.text
        row_data[headers[stat_type]] = stat_value
    
    if row_data:
        data.append(row_data)


Set Streak Averages to Appropriate Variable

In [129]:
last_7_data = data[0]
last_15_data = data[1]
last_30_data = data[2]

last_7_stats = {}
last_15_stats = {}
last_30_stats = {}

last_7_stats['ab_last_7'] = int(last_7_data['AB'])
last_7_stats['h_last_7'] = int(last_7_data['H'])
last_7_stats['r_last_7'] = int(last_7_data['R'])
last_7_stats['rbi_last_7'] = int(last_7_data['RBI'])
last_7_stats['hr_last_7'] = int(last_7_data['HR'])
last_7_stats['bb_last_7'] = int(last_7_data['BB'])
last_7_stats['k_last_7'] = int(last_7_data['SO'])
last_7_stats['ba_last_7'] = float(last_7_data['AVG'])
last_7_stats['obp_last_7'] = float(last_7_data['OBP'])
last_7_stats['slg_last_7'] = float(last_7_data['SLG'])

last_15_stats['ab_last_15'] = int(last_15_data['AB'])
last_15_stats['h_last_15'] = int(last_15_data['H'])
last_15_stats['r_last_15'] = int(last_15_data['R'])
last_15_stats['rbi_last_15'] = int(last_15_data['RBI'])
last_15_stats['hr_last_15'] = int(last_15_data['HR'])
last_15_stats['bb_last_15'] = int(last_15_data['BB'])
last_15_stats['k_last_15'] = int(last_15_data['SO'])
last_15_stats['ba_last_15'] = float(last_15_data['AVG'])
last_15_stats['obp_last_15'] = float(last_15_data['OBP'])
last_15_stats['slg_last_15'] = float(last_15_data['SLG'])

last_30_stats['ab_last_30'] = int(last_30_data['AB'])
last_30_stats['h_last_30'] = int(last_30_data['H'])
last_30_stats['r_last_30'] = int(last_30_data['R'])
last_30_stats['rbi_last_30'] = int(last_30_data['RBI'])
last_30_stats['hr_last_30'] = int(last_30_data['HR'])
last_30_stats['bb_last_30'] = int(last_30_data['BB'])
last_30_stats['k_last_30'] = int(last_30_data['SO'])
last_30_stats['ba_last_30'] = float(last_30_data['AVG'])
last_30_stats['obp_last_30'] = float(last_30_data['OBP'])
last_30_stats['slg_last_30'] = float(last_30_data['SLG'])

last_7_df = pd.DataFrame([last_7_stats])
last_15_df = pd.DataFrame([last_15_stats])
last_30_df = pd.DataFrame([last_30_stats])

streak_data = pd.concat([last_7_df, last_15_df, last_30_df], axis=1)

print(streak_data)

   ab_last_7  h_last_7  r_last_7  rbi_last_7  hr_last_7  bb_last_7  k_last_7  \
0         29         9         3           5          0          3         9   

   ba_last_7  obp_last_7  slg_last_7  ...  ab_last_30  h_last_30  r_last_30  \
0       0.31       0.375       0.345  ...          85         30         19   

   rbi_last_30  hr_last_30  bb_last_30  k_last_30  ba_last_30  obp_last_30  \
0           20           4          10         23       0.353        0.429   

   slg_last_30  
0        0.553  

[1 rows x 30 columns]


Fetch Batter Season Averages

Add Season Averages to Streak Data

Fetch Starting Pitcher Season Data

Format Batter vs Starting Pitcher URL

In [130]:
formatted_batter_name = player_name.replace(' ', '%20')

bvp_url = f'https://baseballsavant.mlb.com/illustrator?playerId={player_id}&playerType=batter&name={formatted_batter_name}&tabSelection=0&shape=square&shapeName=Square%20(Instagram)&subTitle=&chartType=zoneChart-totalPitches&chartName=Zone%20-%20Total%20Pitches&pitcherThrows=&batterStands=&plateAppearanceResults=[]&pitchTypes=[]&seasonType=[%22R%22]&pitchResults=[]&facingPlayer=[%22{sPitcher_id}%22]&balls=[]&battedBallType=[]&years=[{year}]&strikes=[]&outs=[]&selectedGames=[]&homeTeam=[]&awayTeam=[]&vsTeam=[]&exitVelocityGT=0&exitVelocityLT=125&pitchVelocityGT=0&pitchVelocityLT=105&launchAngleGT=-90&launchAngleLT=90&perspective=catcher&venue=7'

Fetch Batter vs Starting Pitcher Stats

In [131]:
bvp_raw_data = {}
bvp_data = {'vs_sp_pa': None,
'vs_sp_p': None,
'vs_sp_h': None,
'vs_sp_hr': None,
'vs_sp_ba': None,
'vs_sp_k': None,
'vs_sp_k_p': None }

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(bvp_url)

driver.implicitly_wait(10)

stats_boxes = driver.find_elements(By.CLASS_NAME, 'stats-box')

for stats_box in stats_boxes:
    stat_type = stats_box.find_element(By.CLASS_NAME, 'stats-title')
    stat_value = stats_box.find_element(By.CLASS_NAME, 'stats-value')
    bvp_raw_data[stat_type.text] = stat_value.text


driver.quit()

bvp_data['vs_sp_pa'] = int(bvp_raw_data['PA'])
bvp_data['vs_sp_p'] = int(bvp_raw_data['Pitches'])
bvp_data['vs_sp_h'] = int(bvp_raw_data['H'])
bvp_data['vs_sp_hr'] = int(bvp_raw_data['HR'])
bvp_data['vs_sp_ba'] = float(bvp_raw_data['AVG'])
bvp_data['vs_sp_k'] = int(bvp_raw_data['SO'])
bvp_data['vs_sp_k_p'] = round((float(bvp_raw_data['K%']) / 100), 3)

bvp_data_df = pd.DataFrame([bvp_data])

print(bvp_data_df)



   vs_sp_pa  vs_sp_p  vs_sp_h  vs_sp_hr  vs_sp_k  vs_sp_k_p  vs_sp_ba
0         3       14        1         0        1      0.333     0.333


Format URL for Batter vs Opp Data

Fetch Batter vs Opp Stats

Fetch Umpire Stats
- If Home Plate Ump announced
    - Return data for only HP ump 
- If Home Plate Ump not yet announced
    - Fetch data for all 4 umps
    - hp_ump stats = avg of 4 umps stats

Fetch Venue Stats *WIP